In [2]:
using Pkg
Pkg.activate("presentation")
Pkg.add(["BenchmarkTools", "MATLAB", "LoopVectorization",
         "MarketData", "TimeSeries", "Distributions",
         "KernelDensity", "StatsPlots", "ProgressMeter",
         "ARCHModels"])
Pkg.update()


  Activating environment at `C:\Users\sabro\Documents\julia\notebooks\ebs\presentation\Project.toml`
    Updating registry at `C:\Users\sabro\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\sabro\Documents\julia\notebooks\ebs\presentation\Project.toml`
  No Changes to `C:\Users\sabro\Documents\julia\notebooks\ebs\presentation\Manifest.toml`
    Updating registry at `C:\Users\sabro\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Installed TimeSeries ─ v0.22.1
    Updating `C:\Users\sabro\Documents\julia\notebooks\ebs\presentation\Project.toml`
  [31c24e10] ↑ Distributions v0.23.12 ⇒ v0.24.18
  [91a5bcdd] ↑ Plots v0.29.9 ⇒ v1.16.2
  [9e3dc215] ↑ TimeSeries v0.20.1 ⇒ v0.22.1
    Updating `C:\Users\sabro\Documents\julia\notebooks\ebs\presentation\Manifest.toml`
  [d360d2e6] + ChainRulesCore v0.10.1
  [35d6a980] + ColorSchemes v3.1

##### <p style="text-align: center; font-size: 300%"> The Unreasonable Effectiveness of Julia </p>
<p style="text-align: center; font-size: 200%"> Simon A. Broda </p>
<p style="text-align: center; font-size: 100%"> University of Amsterdam and Hochschule Luzern<br>
<a href="mailto:simon.broda@uzh.ch">s.a.broda@uva.nl</a> </p>
<img src="EULOGO.jpg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 20%;">
<p style="text-align: center; font-size: 100%"> This project has received funding from the European Research Council (ERC) under the European Union's Horizon 2020 research and innovation program (grant agreement No. 750559). </p>

# Outline
* The Julia Language
* Refresher on ARCH Models
* The `ARCHModels.jl` Package
   * Usage
   * Benchmarks
 

# The Julia Language
## General Information
* New programming language started at MIT.
* Designed with scientific computing in mind.
* Version 1.0 released in August 2018 after 9 years of development.
* Free and open source software. Available at https://julialang.org/.
* In the words of its creators,
> We want a language that’s open source, with a liberal license. We want the speed of C with the dynamism of Ruby. We want a
language that’s homoiconic, with true macros like Lisp, but with obvious, familiar mathematical notation like Matlab. We want
something as usable for general programming as Python, as easy for statistics as R, as natural for string processing as Perl, as
powerful for linear algebra as Matlab, as good at gluing programs together as the shell. Something that is dirt simple to learn,
yet keeps the most serious hackers happy. We want it interactive and we want it compiled.



## Highlights for Scientists
* Interactive REPL (like Matlab, Python, R, etc.) allows for exploratoty analysis, rapid prototyping.
* Unlike these, Julia is JIT compiled, hence fast (typically within a few percent of C)
* Syntax superficially similar to Matlab.
* Rich type system, multiple dispatch.
* Fast-growing eco-system with many state-of-the-art packages (e.g., `ForwardDiff.jl`, `DifferentialEquations.jl`, `JuMP`, ...).


## A Small Taste of Julia

* Let's say we want to implement a function that sums an array:

In [ ]:
function mysum(x)
    s = zero(eltype(x))
    for i in 1:length(x)
        s += x[i]
    end
    s
end
mysum([1., 2., 3.])

* Let's benchmark it:

In [ ]:
x = randn(10^7);

In [ ]:
using BenchmarkTools
@btime mysum(x); # @btime runs the code a large number of times

* For comparison, the built-in function:

In [ ]:
@btime sum(x);

* Close, but we are about 3x slower.
* But we can do better! 2nd attempt:

In [ ]:
function mysum(x)
    s = zero(eltype(x))
    @simd for i in 1:length(x)
        s += x[i]
    end
    s
end
mysum([1., 2., 3.])

In [ ]:
@btime mysum(x);

* Not bad: by just adding a simple decorator, we now match the speed of the built-in function!
* This is, in fact, expected: the built-in function is implemented in Julia, like most of the standard library.
* We can even look at the code:

In [ ]:
@which sum(x)

## One more thing...

In [ ]:
using LoopVectorization
function mysum(x)
    s = zero(eltype(x))
    @avxt for i in 1:length(x)
        s += x[i]
    end
    s
end
mysum([1., 2., 3.])

In [ ]:
@btime mysum(x);

* Wow :o

# Refresher on ARCH Models
* Daily financial returns data exhibit a number of *stylized facts*:
  * Volatility clustering
  * Non-Gaussianity, fat tails
  * Leverage effects: negative returns increase future volatility
* Other types of data (e.g., changes in interest rates) exhibit similar phenomena.
* These effects are important in many areas in finance, in particular in risk management.
* [G]ARCH ([**G**eneralized] **A**utoregressive **C**onditional **V**olatility) models are the most popular for modelling them.

## Example: volatility clustering in AAPL returns
* The `MarketData` package contains several historial datasets. 

In [16]:
using MarketData, TimeSeries
r = percentchange(yahoo(:AAPL)[:AdjClose]) # returns a TimeArray
data = values(r) # an array containing just the plain data

#if !isfile("returns.svg") || !isfile("kde.svg")
    using Distributions, KernelDensity, StatsPlots
    plot(r, title="Volatility Clustering", legend=:none, ylabel="\$r_t\$")
    savefig("returns.svg")
    plot(kde(data), label="Kernel Density", title="Fat Tails")
    plot!(fit(Normal, data), label="Fitted Normal")
    savefig("kde.svg")
#end

<img src="returns.svg" alt="RETURNS" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

<img src="kde.svg" alt="Kernel Density" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">



## (G)ARCH Models
* Basic setup: given a sample of financial returns $\{r_t\}_{t\in\{1,\ldots,T\}}$, decompose $r_t$ as
$$
r_t=\mu_t+\sigma_tz_t, \quad z_t\stackrel{i.i.d.}{\sim}(0,1),
$$
where $\mu_t\equiv\mathbb{E}[r_t\mid \mathcal{F}_{t-1}]$ and $\sigma_t^2\equiv \mathbb{E}[(r_t-\mu_t)^2\mid \mathcal{F}_{t-1}]$.
* Assume $\mu_t=0$ for simplicity. Focus is on the *volatility* $\sigma_t$. G(ARCH) models make $\sigma_t$ a function of *past* returns and variances. Examples: 

## Examples
* ARCH(q) (Engle, Econometrica 1982):
$$\sigma_t^2=\omega+\sum_{i=1}^q \alpha_ir_{t-i}^2%,\quad \omega,\alpha_i>0,\quad \sum_{i=1}^q\alpha_i<1.
$$
* GARCH(p, q) (Bollerslev, JoE 1986)
$$\sigma_t^2=\omega+ \sum_{i=1}^p\beta_{i}\sigma_{t-i}^2 + \sum_{i=1}^q\alpha_ir_{t-i}^2%,\quad \omega,\alpha_i,\beta_i>0,\quad \sum_{i=1}^{\max p,q} \alpha_i+\beta_i<1.
$$
* EGARCH(o, p, q) (Nelson, Econometrica 1991)
$$\log(\sigma_t^2)=\omega+\sum_{i=1}^o\gamma_{i}z_{t-i}+\sum_{i=1}^p\beta_i\log(\sigma_{t-i}^2)+\sum_{i=1}^q \alpha_i (|z_t|-\mathbb{E}|z_t|)%, \quad \sum_{i=1}^p \beta_i<0.
$$
 
  



## Estimation
* G(ARCH) models are usually estimated by maximum likelihood: with $f_z$ denoting the density of $z_t$,
$$\max \sum_t \log f(r_t\mid \mathcal{F}_{t-1})=\max \sum_t \log f_z(r_t/\sigma_t)-\log \sigma_t.$$
* Recursive nature of $\sigma_t$ means the computation cannot be "vectorized" $\Rightarrow$ loops.
* Julia is very well suited for this. Matlab (and the `rugarch` package for Python) have to implement the likelihood in C.



# The `ARCHModels.jl` Package
## Installation
* `ARCHModels.jl` is a registered julia package. To install it, just do `using Pkg; Pkg.add("ARCHModels")`
*  Extensive documentation available at https://s-broda.github.io/ARCHModels.jl/stable/.
* `ARCH.jl` is not a registered Julia package yet. To install it in Julia 1.0 or later, do

## Key Features
* Supports simulating, estimating, forecasting, and backtesting ARCH models.
* Currently: ARCH, GARCH, TGARCH, and EGARCH models of arbitrary orders, with Gaussian, Student's $t$, and GED errors.
* Regression and ARMA models for the mean equation.
* CCC and DCC models for multivariate returns.
* Entirely written in Julia.
* Designed to be easily extensible with new models and distributions (nice thesis project...)
* Gradients and Hessians (for both numerical maximization of the likelihood and constructing standard errors) are obtained by [automatic differentiation](http://www.autodiff.org/?module=Introduction) via `ForwardDiff.jl`.

## Usage

* The first step in building an ARCH model is usually to test for the presence of volatility clustering.
* `ARCHModels.jl` provides [Engle's (Econometrica 1982)](https://doi.org/10.2307/1912773) ARCH-LM test for this.
* The null is that $\gamma_i=0$ in the auxiliary regression
$$
r_t^2=\alpha+\gamma_1 r_{t-1}^2+\gamma_2 r_{t-2}^2+\cdots+\gamma_p r_{t-p}^2+\epsilon_t.
$$

In [17]:
using ARCHModels
ARCHLMTest(data, 4) # p=4 lags

ARCH LM test for conditional heteroskedasticity
-----------------------------------------------
Population details:
    parameter of interest:   T⋅R² in auxiliary regression
    value under h_0:         0
    point estimate:          147.398

Test summary:
    outcome with 95% confidence: reject h_0
    p-value:                     <1e-30

Details:
    sample size:                    10205
    number of lags:                 4
    LM statistic:                   147.398


* Unsurprisingly, the test rejects.

* Fitting a GARCH model is as simple as

In [18]:
model = fit(GARCH{1, 1}, data)


GARCH{1, 1} model with Gaussian errors, T=10205.

Mean equation parameters:
─────────────────────────────────────────────
     Estimate    Std.Error  z value  Pr(>|z|)
─────────────────────────────────────────────
μ  0.00184929  0.000257205  7.18994    <1e-12
─────────────────────────────────────────────

Volatility parameters:
─────────────────────────────────────────────
      Estimate  Std.Error   z value  Pr(>|z|)
─────────────────────────────────────────────
ω   5.67925e-6  2.9651e-6   1.91537    0.0554
β₁  0.929851    0.0136741  68.0008     <1e-99
α₁  0.0674646   0.0176432   3.82384    0.0001
─────────────────────────────────────────────


* Alternatively, the (multithreaded!) `selectmodel` method does automatic model selection (i.e., chooses $p$ and $q$ by minimizing the AIC/AICC/BICC).

In [19]:
model2 = selectmodel(GARCH, data; maxlags=3, criterion=bic) # optional keyword arguments


GARCH{3, 1} model with Gaussian errors, T=10205.

Mean equation parameters:
─────────────────────────────────────────────
     Estimate    Std.Error  z value  Pr(>|z|)
─────────────────────────────────────────────
μ  0.00201148  0.000241287  8.33647    <1e-16
─────────────────────────────────────────────

Volatility parameters:
──────────────────────────────────────────────────
       Estimate   Std.Error      z value  Pr(>|z|)
──────────────────────────────────────────────────
ω   1.34075e-5   1.42247e-6  9.42551        <1e-20
β₁  0.520858     0.088458    5.88819        <1e-08
β₂  1.36846e-48  3.42138e-7  3.99972e-42    1.0000
β₃  0.358345     0.0886664   4.0415         <1e-04
α₁  0.101772     0.0134813   7.54909        <1e-13
──────────────────────────────────────────────────


* The return value is of type `ARCHModel`:

In [20]:
typeof(model)

UnivariateARCHModel{Float64, GARCH{1, 1, Float64}, StdNormal{Float64}, Intercept{Float64}}

* `UnivariateARCHModel` supports many useful methods, such as 
 `confint`, `aic`, `bic`, `aicc`, `informationmatrix`, `score`, `vcov`:


In [22]:
aic(model)

-45895.13689151763

* A `UnivariateARCHModel` essentially consists of a volatility specification (of type `VolatilitySpec`),
an error distribution (of type `StandardizedDistribution`), and a mean specification (of type `MeanSpec`).
* The following are currently implemented:

In [26]:
print.(string.(subtypes(UnivariateVolatilitySpec)) .* " ");

EGARCH TGARCH 

In [24]:
print.(string.(subtypes(StandardizedDistribution))[2:end] .* " ");

StdGED StdNormal StdSkewT StdT 

In [25]:
print.(string.(subtypes(MeanSpec)).* " ");

ARMA Intercept NoIntercept Regression 

* We can use these to construct an ARCHModel manually:

In [29]:
T = 10^4
mydata = zeros(T)
volaspec = EGARCH{1, 1, 1}([0.02, .09, .83, .01])
using Random; Random.seed!(1)
mymodel = UnivariateARCHModel(volaspec, mydata; dist=StdT(4.)) # dist is an optional keyword argument


EGARCH{1, 1, 1} model with Student's t errors, T=10000.


─────────────────────────────────────────────────
                              ω    γ₁    β₁    α₁
─────────────────────────────────────────────────
Volatility parameters:     0.02  0.09  0.83  0.01
─────────────────────────────────────────────────
──────────────────────────────
                             ν
──────────────────────────────
Distribution parameters:   4.0
──────────────────────────────


* With a `UnivariateARCHModel` in hand, we can do things like

In [30]:
simulate!(mymodel) # by convention, the bang indicates that the method modifies its argument


EGARCH{1, 1, 1} model with Student's t errors, T=10000.


─────────────────────────────────────────────────
                              ω    γ₁    β₁    α₁
─────────────────────────────────────────────────
Volatility parameters:     0.02  0.09  0.83  0.01
─────────────────────────────────────────────────
──────────────────────────────
                             ν
──────────────────────────────
Distribution parameters:   4.0
──────────────────────────────


In [31]:
fit!(mymodel)


EGARCH{1, 1, 1} model with Student's t errors, T=10000.


Volatility parameters:
───────────────────────────────────────────────
       Estimate   Std.Error   z value  Pr(>|z|)
───────────────────────────────────────────────
ω    0.0274054   0.00733817   3.73464    0.0002
γ₁   0.113959    0.0131327    8.67752    <1e-17
β₁   0.811765    0.0244774   33.1638     <1e-99
α₁  -0.00637894  0.0153358   -0.41595    0.6774
───────────────────────────────────────────────

Distribution parameters:
─────────────────────────────────────────
   Estimate  Std.Error  z value  Pr(>|z|)
─────────────────────────────────────────
ν   3.83768   0.152659   25.139    <1e-99
─────────────────────────────────────────


In [32]:
ARCHLMTest(mymodel) # tests the standardized residuals

ARCH LM test for conditional heteroskedasticity
-----------------------------------------------
Population details:
    parameter of interest:   T⋅R² in auxiliary regression
    value under h_0:         0
    point estimate:          0.0873996

Test summary:
    outcome with 95% confidence: fail to reject h_0
    p-value:                     0.7675

Details:
    sample size:                    10000
    number of lags:                 1
    LM statistic:                   0.0873996


In [33]:
predict(mymodel, :volatility) # (:volatility | :variance | :return | :VaR)

0.9582988646322497

## Value at Risk

* In-sample Value-at-Risk estimates are available via the `VaRs` function

In [34]:
model = fit(GARCH{1, 1}, data)
vars = VaRs(model, 0.05)

if !isfile("VaRplot.svg")
    using Plots
    plot(-data, legend=:none, xlabel="\$t\$", ylabel="\$-r_t\$", title="Value at Risk, In-Sample")
    plot!(vars, color=:purple)
    savefig("VaRplot.svg"); 
end

<img src="VaRplot.svg" alt="VALUE AT RISK" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

* How about out-of-sample (backtesting)?
* Requires re-estimating the model at each time step.
* Not built in, but easy to do:

In [35]:
using ProgressMeter

T = length(data)
windowsize = 1000
vars = similar(data); fill!(vars, NaN)
@showprogress "Fitting $(T-1-windowsize) GARCH models: " for t = windowsize+1:T-1
    m = fit(GARCH{1, 1}, data[t-windowsize:t])
    vars[t+1] = predict(m, :VaR; level=0.05)    
end

if !isfile("VaRplot_oos.svg")
    using Plots
    plot(-data, legend=:none, xlabel="\$t\$", ylabel="\$-r_t\$", title="Value at Risk, Out-of-Sample")
    plot!(vars, color=:purple)
    savefig("VaRplot_oos.svg"); 
end

Fitting 9204 GARCH models: 100%|████████████████████████| Time: 0:00:27


<img src="VaRplot_oos.svg" alt="VALUE AT RISK OUT OF SAMPLE" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

* We can backtest these out-of-sample VaR predictions using Engle and Manganelli's (2004) dynamic quantile test.
* The test is based on a *hit series* 
$$I_t\equiv\cases{1, r_t<-VaR_t\\0, \mbox{otherwise.}}$$
* The null is that $\alpha=\beta=\gamma=0$ in the auxiliary regression
$$
I_t-0.05 = \alpha +\beta I_{t-1} + \gamma VaR_t+\epsilon_t.
$$

In [36]:
DQTest(data[windowsize+1:end], vars[windowsize+1:end], 0.05)

Engle and Manganelli's (2004) DQ test (out of sample)
-----------------------------------------------------
Population details:
    parameter of interest:   Wald statistic in auxiliary regression
    value under h_0:         0
    point estimate:          35.834

Test summary:
    outcome with 95% confidence: reject h_0
    p-value:                     <1e-07

Details:
    sample size:                    9205
    number of lags:                 1
    VaR level:                      0.05
    DQ statistic:                   35.834


# Benchmarks
  * Bollerslev and Ghysels (JBES 1996) data is de facto standard in comparing implementations of GARCH models.
  * Data consist of daily German mark/British pound exchange rates (1974 observations).
  * Available in `ARCHModels.jl` as the constant `BG96`.

In [37]:
if !isfile("DMGBP.svg")
    using Plots
    plot(BG96, legend=:none, title="Bollerslev and Ghysels (1996) Data", ylabel="\$r_t\$", xlabel="\$t\$")
    savefig("DMGBP.svg")
end

<img src="DMGBP.svg" alt="Bollerslev and Ghysels (1996) data" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

## GARCH
* Fitting in Julia:

In [39]:
using BenchmarkTools
@btime fit(GARCH{1, 1}, $BG96, meanspec=NoIntercept)  # Matlab doesn't use an intercept, so let's not, either

  3.099 ms (2267 allocations: 168.53 KiB)



GARCH{1, 1} model with Gaussian errors, T=1974.


Volatility parameters:
─────────────────────────────────────────────
     Estimate   Std.Error   z value  Pr(>|z|)
─────────────────────────────────────────────
ω   0.0108661  0.00657261   1.65324    0.0983
β₁  0.804431   0.0730161   11.0172     <1e-27
α₁  0.154597   0.0539139    2.86747    0.0041
─────────────────────────────────────────────


* Now Matlab:

In [41]:
using MATLAB
mat"version"

LoadError: MATLAB.MEngineError("failed to open MATLAB engine session")

In [ ]:
# run this cell a few times to give Matlab a fair chance
mat"tic; estimate(garch(1, 1), $BG96); toc; 0";

* ARCH.jl is faster by a factor of about 20-30, depending on the machine, despite Matlab calling into compiled C code.
* Estimates are quite similar, but standard errors and $t$-statistics differ.
* So which standard errors are correct? Let's compare with the results from Brooks et. al. (Int. J. Fcst. 2001).

* Brooks et. al. compare implementations of the GARCH(1, 1) model. They use a model with intercept, so let’s re-estimate in Julia (Matlab doesn't seem to allow this):

In [ ]:
fit(GARCH{1, 1}, BG96)

* Brooks et. al. give the estimates (**$t$-stats**)
$\mu=−0.00619$ $(\mathbf{−0.67})$, $\omega=0.0108$ $(\mathbf{1.66})$, $\beta_1=0.806$ $(\mathbf{11.11})$, $\alpha_1=0.153$ $(\mathbf{2.86})$. Dead on!

## EGARCH
* Julia:

In [ ]:
@btime fit(EGARCH{1, 1, 1}, $BG96, meanspec=NoIntercept)

* Matlab:

In [ ]:
mat"tic; estimate(egarch(1, 1), $BG96); toc; 0";  # Matlab sets o=q

* Brooks et. al. give no benchmark results. But again, Julia is faster by a factor of about 20.

#  TODO
* More distributions (NIG, $\alpha$-Stable, ...)
* More GARCH models (APARCH, RiskMetrics, IGARCH, ...)
* Multivariate GARCH


# References
* Bollerslev, T (1986). Generalized autoregressive conditional heteroskedasticity. *Journal of Econometrics* **31**, 307–327.
* Bollerslev, T. & Ghysels, E. (1996). Periodic Autoregressive Conditional Heteroscedasticity. *Journal of Business & Economic Statistics* **14**, 139-151. https://doi.org/10.1080/07350015.1996.10524640.
* Brooks, C., Burke, S. P., & Persand, G. (2001). Benchmarks and the accuracy of GARCH model estimation. *International Journal of Forecasting* **17**, 45-56. https://doi.org/10.1016/S0169-2070(00)00070-4.
* Engle, R. F. (1982). Autoregressive Conditional Heteroscedasticity with Estimates of the Variance of United Kingdom Inflation. *Econometrica* **50**, 987-1007. https://doi.org/10.2307/1912773.
* Engle, R. F., and Manganelli, S. (2004). CAViaR: Conditional Autoregressive Value at Risk by Regression Quantiles. *Journal of Business & Economic Statistics* **22**, 367-381. https://doi.org/10.1198/073500104000000370
* Nelson, D.B. (1991). Conditional Heteroskedasticity in Asset Returns: A New Approach. *Econometrica* **59**, 347--370. https://doi.org/10.2307/2938260.

